# Regression Week 2: Multiple Regression (Interpretation)

The goal of this first notebook is to explore multiple regression and feature engineering with existing graphlab functions.

In this notebook you will use data on house sales in King County to predict prices using multiple regression. You will:
* Use SFrames to do some feature engineering
* Use built-in graphlab functions to compute the regression weights (coefficients/parameters)
* Given the regression weights, predictors and outcome write a function to compute the Residual Sum of Squares
* Look at coefficients and interpret their meanings
* Evaluate multiple models via RSS

# Import libraries

In [9]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as sm

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [52]:
train_data = pd.read_csv('../data/kc_house_train_data.csv')
test_data = pd.read_csv('../data/kc_house_test_data.csv')

# Learning a multiple regression model

Recall we can use the following code to learn a multiple regression model predicting 'price' based on the following features:
example_features = ['sqft_living', 'bedrooms', 'bathrooms'] on training data with the following code:

(Aside: We set validation_set = None to ensure that the results are always the same)

In [53]:
example_model = sm.ols(formula= "price ~ sqft_living + bedrooms + bathrooms", data = train_data).fit()

In [54]:
example_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.510
Model:                            OLS   Adj. R-squared:                  0.510
Method:                 Least Squares   F-statistic:                     6039.
Date:                Sun, 24 Sep 2017   Prob (F-statistic):               0.00
Time:                        05:03:43   Log-Likelihood:            -2.4133e+05
No. Observations:               17384   AIC:                         4.827e+05
Df Residuals:                   17380   BIC:                         4.827e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept    8.791e+04   7873.338     11.166      0.000    7.25e+04    1.03e+05
sqft_living   315.4067      3.456     91.271      0.000     308.633     322.180
bedrooms    -6.508e+04   2717.457    -23.950      0.000   -7.04e+04   -5.98e+04
bathrooms    6942.1660   3923.115      1.770      0.077    -747.533    1.46e+04
==============================================================================
Omnibus:                    11380.189   Durbin-Watson:                   1.998
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           366238.275
Skew:                           2.672   Prob(JB):                         0.00
Kurtosis:                      24.842   Cond. No.                     9.48e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 9.48e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Now that we have fitted the model we can extract the regression weights (coefficients) as follows:

In [55]:
example_weight_summary = example_model.params
print (example_weight_summary)

Intercept      87912.865815
sqft_living      315.406691
bedrooms      -65081.887116
bathrooms       6942.165986
dtype: float64


# Making Predictions

In the gradient descent notebook we use numpy to do our regression. In this book we will use existing graphlab create functions to analyze multiple regressions. 

Recall that once a model is built we can use the .predict() function to find the predicted values for data we pass. For example using the example model above:

In [56]:
example_predictions = example_model.predict(sales_train)
print (example_predictions[0]) # should be 271789.505878

271789.26538


# Compute RSS

Now that we can make predictions given the model, let's write a function to compute the RSS of the model. Complete the function below to calculate RSS given the model, data, and the outcome.

In [57]:
def get_residual_sum_of_squares(model, data, outcome):
    # First get the predictions
    predict = model.predict(data)
    # Then compute the residuals/errors
    resid = outcome - predict
    # Then square and add them up
    RS = resid ** 2
    RSS = RS.sum()
    return(RSS)    

Test your function by computing the RSS on TEST data for the example model:

In [58]:
rss_example_train = get_residual_sum_of_squares(example_model, sales_test, sales_test['price'])
print (rss_example_train) # should be 2.7376153833e+14

2.73761940583e+14


# Create some new features

Although we often think of multiple regression as including multiple different features (e.g. # of bedrooms, squarefeet, and # of bathrooms) but we can also consider transformations of existing features e.g. the log of the squarefeet or even "interaction" features such as the product of bedrooms and bathrooms.

You will use the logarithm function to create a new feature. so first you should import it from the math library.

In [50]:
from math import log

Next create the following 4 new features as column in both TEST and TRAIN data:
* bedrooms_squared = bedrooms\*bedrooms
* bed_bath_rooms = bedrooms\*bathrooms
* log_sqft_living = log(sqft_living)
* lat_plus_long = lat + long 
As an example here's the first one:

In [59]:
train_data['bedrooms_squared'] = train_data['bedrooms'].apply(lambda x: x**2)
test_data['bedrooms_squared'] = test_data['bedrooms'].apply(lambda x: x**2)

# create the remaining 3 features in both TEST and TRAIN data

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this feature will mostly affect houses with many bedrooms.
* bedrooms times bathrooms gives what's called an "interaction" feature. It is large when *both* of them are large.
* Taking the log of squarefeet has the effect of bringing large values closer together and spreading out small values.
* Adding latitude to longitude is totally non-sensical but we will do it anyway (you'll see why)

**Quiz Question: What is the mean (arithmetic average) value of your 4 new features on TEST data? (round to 2 digits)**

In [80]:
train_data['bed_bath_rooms'] = train_data['bedrooms'] * train_data['bathrooms']
test_data['bed_bath_rooms'] = test_data['bedrooms'] * test_data['bathrooms']

train_data['log_sqft_living'] = train_data['sqft_living'].apply(lambda x: log(x))
test_data['log_sqft_living'] = test_data['sqft_living'].apply(lambda x: log(x))

train_data['lat_plus_long'] = train_data['lat'] + train_data['long']
test_data['lat_plus_long'] = test_data['lat'] + test_data['long']

In [87]:
test_data.loc[:,['bedrooms_squared', 'bed_bath_rooms', 'log_sqft_living', 'lat_plus_long']].mean()

bedrooms_squared    12.446678
bed_bath_rooms       7.503902
log_sqft_living      7.550275
lat_plus_long      -74.653334
dtype: float64

# Learning Multiple Models

Now we will learn the weights for three (nested) models for predicting house prices. The first model will have the fewest features the second model will add one more feature and the third will add a few more:
* Model 1: squarefeet, # bedrooms, # bathrooms, latitude & longitude
* Model 2: add bedrooms\*bathrooms
* Model 3: Add log squarefeet, bedrooms squared, and the (nonsensical) latitude + longitude

In [ ]:
model_1_features = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']
model_2_features = model_1_features + ['bed_bath_rooms']
model_3_features = model_2_features + ['bedrooms_squared', 'log_sqft_living', 'lat_plus_long']

Now that you have the features, learn the weights for the three different models for predicting target = 'price' using graphlab.linear_regression.create() and look at the value of the weights/coefficients:

In [90]:
# Learn the three models: (don't forget to set validation_set = None)
model_1 = sm.ols(formula= "price ~ sqft_living + bedrooms + bathrooms + lat + long", data = train_data).fit()
model_2 = sm.ols(formula= "price ~ sqft_living + bedrooms + bathrooms + lat + long + bed_bath_rooms", 
                 data = train_data).fit()
model_3 = sm.ols(formula= "price ~ sqft_living + bedrooms + bathrooms + lat + long + bed_bath_rooms + bedrooms_squared + log_sqft_living + lat_plus_long", 
                 data = train_data).fit()

In [94]:
# Examine/extract each model's coefficients:
print("model_1\n", model_1.params)
print("\nmodel_2\n", model_2.params)
print("\nmodel_3\n", model_3.params)

model_1
 Intercept     -6.907573e+07
sqft_living    3.122586e+02
bedrooms      -5.958653e+04
bathrooms      1.570674e+04
lat            6.586193e+05
long          -3.093744e+05
dtype: float64

model_2
 Intercept        -6.686797e+07
sqft_living       3.066101e+02
bedrooms         -1.134464e+05
bathrooms        -7.146131e+04
lat               6.548446e+05
long             -2.942990e+05
bed_bath_rooms    2.557965e+04
dtype: float64

model_3
 Intercept          -6.203608e+07
sqft_living         5.294228e+02
bedrooms            3.451423e+04
bathrooms           6.706078e+04
lat                 5.340856e+05
long               -4.067507e+05
bed_bath_rooms     -8.570504e+03
bedrooms_squared   -6.788587e+03
log_sqft_living    -5.618315e+05
lat_plus_long       1.273349e+05
dtype: float64


**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 1?**

**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 2?**

Think about what this means.

# Comparing multiple models

Now that you've learned three models and extracted the model weights we want to evaluate which model is best.

First use your functions from earlier to compute the RSS on TRAINING Data for each of the three models.

In [99]:
# Compute the RSS on TRAINING data for each of the three models and record the values:
rss_model1_train = get_residual_sum_of_squares(model_1, train_data, train_data['price'])
rss_model2_train = get_residual_sum_of_squares(model_2, train_data, train_data['price'])
rss_model3_train = get_residual_sum_of_squares(model_3, train_data, train_data['price'])

print ("RSS of model_1: ", rss_model1_train) 
print ("RSS of model_2: ", rss_model2_train) 
print ("RSS of model_3: ", rss_model3_train) 

RSS of model_1:  9.6787996305e+14
RSS of model_2:  9.58419635074e+14
RSS of model_3:  9.0343645505e+14


**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TRAINING Data?** Is this what you expected?

Now compute the RSS on on TEST data for each of the three models.

In [100]:
# Compute the RSS on TESTING data for each of the three models and record the values:
rss_model1_test = get_residual_sum_of_squares(model_1, test_data, test_data['price'])
rss_model2_test = get_residual_sum_of_squares(model_2, test_data, test_data['price'])
rss_model3_test = get_residual_sum_of_squares(model_3, test_data, test_data['price'])

print ("RSS of model_1: ", rss_model1_test) 
print ("RSS of model_2: ", rss_model2_test) 
print ("RSS of model_3: ", rss_model3_test) 

RSS of model_1:  2.25500469795e+14
RSS of model_2:  2.23377462976e+14
RSS of model_3:  2.59236319207e+14


**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TESTING Data?** Is this what you expected? Think about the features that were added to each model from the previous.